In [1]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader, 
    StorageContext,
    load_index_from_storage,
    Document,
    PromptTemplate
)
load_dotenv()

True

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
Settings.embed_model = embed_model

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [3]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [4]:
import os
PERSIST_DIR = "lyrics_store2"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("47923_BTS.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [5]:
query_engine = index.as_query_engine()

In [6]:
qa_template_str = """
    You are an expert in K-Pop music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)

In [7]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [8]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

description = """
A set of lyrics for songs from the Korean Group BTS. 
Use plain text question as input to the tool. 
MANDATORY: Pass the response to the user as is, mantaining the format, do not try to summarize when using this tool.
"""
tools_lyrics = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="BTS_songs_lyrics",
            description=description,
            return_direct=False
        )
    )
]
agent = OpenAIAgent.from_tools(tools=tools_lyrics, verbose=True)

In [9]:
response = agent.chat("¿Qué canciones de BTS hablan de nostalgia?")

Added user message to memory: ¿Qué canciones de BTS hablan de nostalgia?
=== Calling Function ===
Calling function: BTS_songs_lyrics with args: {"input":"nostalgia"}
Got output: - En la canción "Bad Decisions" de BTS, se puede percibir un tono de nostalgia en las siguientes líneas:
  - "And it feels like (Ooh, ah) I can't seem to ever get you out of my mind"
  - "I want you, ooh, baby, all of the time"
- La canción expresa un sentimiento de añoranza y deseo constante por la persona amada, reflejando una sensación de nostalgia en la letra.

=== Calling Function ===
Calling function: BTS_songs_lyrics with args: {"input":"nostalgia"}
Got output: - La canción "Bad Decisions" de BTS no aborda directamente el tema de la nostalgia, pero sí habla sobre el deseo de amor y la toma de decisiones arriesgadas en el presente.
- En la canción, se mencionan versos como "And it feels like I can't seem to ever get you out of my mind" y "I want you, ooh, baby, all of the time", que reflejan un sentimient

In [10]:
response.response

'La canción de BTS que habla de nostalgia es "Bad Decisions". En esta canción se puede percibir un tono de añoranza y deseo constante por la persona amada. Algunas líneas que reflejan este sentimiento son: "And it feels like I can\'t seem to ever get you out of my mind" y "I want you, ooh, baby, all of the time".'

In [11]:
response = agent.chat("¿Qué canciones hablan sobre el amor propio?")

Added user message to memory: ¿Qué canciones hablan sobre el amor propio?
=== Calling Function ===
Calling function: BTS_songs_lyrics with args: {"input":"self-love"}
Got output: - La canción "Outro: Luv In Skool" de BTS aborda el tema del amor propio y la autoaceptación.
- En la letra se mencionan frases como "I just wanna love ya" y "Me amo a mí mismo", que reflejan la importancia de valorarse a uno mismo.
- Se hace referencia a la idea de conocerse a uno mismo y amarse tal como se es.
- El tono de la canción es positivo y empoderador, animando a los oyentes a amarse a sí mismos y a no dejarse influenciar por los haters.

=== Calling Function ===
Calling function: BTS_songs_lyrics with args: {"input":"self-love"}
Got output: - La canción "Outro: Luv In Skool" de BTS aborda el tema del amor propio y la autoaceptación.
- En la letra se mencionan frases como "I just wanna love ya" y "Me amo a mí mismo", que reflejan la importancia de valorarse a uno mismo.
- El verso "Me amo, me amo, me

### TAREA 15-10-2024

In [12]:
import wikipedia
from llama_index.core import Document, VectorStoreIndex
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
def get_artist_documents(filename: str) -> list[Document]:
  with open(filename) as file:
    data = file.read()
  songs = data.split("===")
  artist = songs.pop(0).strip()
  
  documents = [
    Document(
      text=song,
      metadata={
        "category":"music",
        "artist": artist,
      }
    )
    for song in songs
  ]    
  return documents

In [14]:
bts_songs_file = '47923_BTS.txt'

lyrics_gotten = get_artist_documents(bts_songs_file)

In [15]:
wikipedia.set_lang('es')

bts_content = wikipedia.page('BTS').content

with open('BTS_info.txt', 'w', encoding='utf-8') as file:
  file.write("== Nombre ==\n")
  file.write(bts_content)

In [16]:
def get_bts_documents(filename: str) -> list[Document]:
  with open(filename, 'r', encoding='utf-8') as file:
    data = file.read()
  
  sections = data.split("===")
  sections = [section.strip() for section in sections if section.strip()]
  
  artist_line = sections.pop(0)
  artist = artist_line.split("==")[1].strip() if "==" in artist_line else "BTS"
  
  documents = [
    Document(
      text=section.strip(),
      metadata={
        "category": "music",
        "artist": artist
      }
    )
    for section in sections
  ]
  
  return documents

In [17]:
bts_info_documents = get_bts_documents('BTS_info.txt')

index = VectorStoreIndex.from_documents(bts_info_documents)

query_engine = index.as_query_engine()

In [18]:
description = """
This tool provides information about BTS, including song lyrics and biographical data obtained from Wikipedia.
It accepts natural language questions as input.
MANDATORY: Provide the response to the user exactly as it is, preserving the format; do not attempt to summarize when using this tool.
"""

In [19]:
tools_wiki = [
  QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
      name="BTS_Info_and_Lyrics",
      description=description,
      return_direct=False
    )
  )
]

bts_agent = OpenAIAgent.from_tools(tools=tools_wiki + tools_lyrics, verbose=True)

In [20]:
response = bts_agent.chat('¿Puedes describir los temas líricos de BTS?')
print(response)

response = bts_agent.chat('¿Cual es el origen de BTS?')
print(response)

Added user message to memory: ¿Puedes describir los temas líricos de BTS?
=== Calling Function ===
Calling function: BTS_Info_and_Lyrics with args: {"input":"Describe the lyrical themes of BTS"}
Got output: The lyrical themes of BTS often revolve around self-reflection, mental health, youth empowerment, social issues, and the journey of self-discovery. Their songs frequently touch on topics like personal growth, following dreams, love, friendship, and the pressures faced by young people in today's society.

Los temas líricos de BTS suelen girar en torno a la autorreflexión, la salud mental, el empoderamiento de la juventud, problemas sociales y el viaje de autodescubrimiento. Sus canciones a menudo abordan temas como el crecimiento personal, seguir los sueños, el amor, la amistad y las presiones que enfrentan los jóvenes en la sociedad actual.
Added user message to memory: ¿Cual es el origen de BTS?
=== Calling Function ===
Calling function: BTS_Info_and_Lyrics with args: {"input":"Ori

In [21]:
import gradio as gr

def very_wise_response(message, history):
  return bts_agent.chat(message).response

demo = gr.ChatInterface(very_wise_response, type="messages")

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Added user message to memory: Puedes darme un resumen de la historia del origen de BTS?
=== Calling Function ===
Calling function: BTS_Info_and_Lyrics with args: {"input":"Summary of the origin story of BTS"}
Got output: BTS, a music group consisting of seven members, originated in South Korea. The group's journey began with their debut in 2013 under Big Hit Entertainment. Over the years, BTS has gained international recognition for their music, performances, and philanthropic efforts. Their discography includes albums like "Dark & Wild," "Love Yourself: Tear," and "Map of the Soul: 7." BTS has embarked on various successful tours worldwide, such as "The Wings Tour" and "Love Yourself World Tour." The group's popularity continues to grow, making them a global phenomenon in the music industry.

